In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import pcraster as pcr
from osgeo import gdal
import glob
import netCDF4 as nc
from mpl_toolkits.basemap import Basemap
import datetime as dt
from osgeo import gdal
import rasterio
from rasterio.plot import show
import spotpy
import pyodbc
import shutil
import pyhecdss #only for environment with Py3.7.11

In [2]:
os.chdir("D:/DANI/2021/TEMA4_PRONOSTICOS/")
os.listdir()

['CN', 'DATOS', 'DOCS', 'EVA', 'GIS', 'HMS', 'PYR', 'REPORTE', 'RTS', 'SSP']

In [3]:
#Import csv data to hecdss
ensemble = [0,1,2,3,4,5,6,7,8,9]
slt = [*range(2,17)]
variables = ['P_228228', 'Rl_169-175', 'Rs_169-175', 'T_121-122', 'Ww_165-166']
vars_dss = {'P_228228':'PRECIP-INC', 'Rl_169-175':'RADIATION-LONG', 'Rs_169-175':'RADIATION-SHORT', 
            'T_121-122':'TEMPERATURE', 'Ww_165-166':'WINDSPEED'}
units_dss = {'P_228228':'MM', 'Rl_169-175':'WATT/M2', 'Rs_169-175':'WATT/M2', 
            'T_121-122':'DEG C', 'Ww_165-166':'M/S'}
type_dss = {'P_228228':'PER-CUM', 'Rl_169-175':'PER-AVER', 'Rs_169-175':'PER-AVER', 
            'T_121-122':'PER-AVER', 'Ww_165-166':'INST-VAL'}

for lt in slt:
    # start_lt = 17
    start_lt = lt
    end_lt = start_lt + 3
    print('lt', start_lt, end_lt)

    fname=r'reforecasts'+str(start_lt)+str(end_lt)+'.dss'
    d = pyhecdss.DSSFile('./PYR/HMS/DSS/'+fname, create_new=True)

    for var in variables:
        
        #control forecasts
        data = pd.read_csv('./DATOS/ECMWF/nc/csv/wa/c_'+var+'_lt'+str(start_lt)+str(end_lt)+'_wa.csv', parse_dates=['Unnamed: 0'])
        data.index = data['Unnamed: 0']
        data = data.drop(['Unnamed: 0'], axis=1)
        data.index.name = None
        data.index.freq = '1D'
        
#         for ens in ensemble:
        Dataset = 'ECMWF-REF-ENS-00' #input('Dataset, A ')
        Location = 'RIOLASILLA' #input('Location, B ')
        Variable = vars_dss[var] #input('Variable, C ')
        Timestep = '1Day' #input('Timestep, E ')
        Units = units_dss[var] #input('Units ')
        Type = type_dss[var] #input('Type' )
        Comments = 'LT '+str(start_lt).zfill(2)+'-'+str(end_lt).zfill(2) #input('Comments, F ')
#             path = '/SAMPLE/ITS1/RANDOM//IR-YEAR/SAMPLE2/'
        path = '/'+Dataset+'/'+Location+'/'+Variable+'//'+Timestep+'/'+Comments+'/'
        
        data_dss = data['c']

        d.write_rts(path, data_dss, Units, Type)
        
        #perturbed forecasts
        data = pd.read_csv('./DATOS/ECMWF/nc/csv/wa/'+var+'_lt'+str(start_lt)+str(end_lt)+'_wa.csv', parse_dates=['Unnamed: 0'])
        data.index = data['Unnamed: 0']
        data = data.drop(['Unnamed: 0'], axis=1)
        data.index.name = None
        data.index.freq = '1D'

        for ens in ensemble:
            Dataset = 'ECMWF-REF-ENS-'+str(ens+1).zfill(2) #input('Dataset, A ')
            path = '/'+Dataset+'/'+Location+'/'+Variable+'//'+Timestep+'/'+Comments+'/'
            data_dss = data[str(ens+1)]

            d.write_rts(path, data_dss, Units, Type)

    d.close()

lt 2 5
lt 3 6
lt 4 7
lt 5 8
lt 6 9
lt 7 10
lt 8 11
lt 9 12
lt 10 13
lt 11 14
lt 12 15
lt 13 16
lt 14 17
lt 15 18
lt 16 19


## Analysis of Results